Read in MWDD tables and usw them to convet Odette's measurements into accretion rates.

In [179]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import os
import glob
from astropy.table import Table
from astropy.io import ascii
import astropy.units as u
import astropy.constants as const
from scipy.interpolate import interpolate
from craftroom import resample
from astropy.convolution import convolve, Box1DKernel
from scipy.io.idl import readsav


#matplotlib set up
%matplotlib inline
from matplotlib import rcParams
rcParams["figure.figsize"] = (14, 5)
rcParams["font.size"] = 20


In [180]:
wdtab = Table.read('tables/pceb_fit_results.csv')

In [181]:
mwdd = glob.glob('mwdd_params/*')

In [182]:
#np.genfromtxt(mwdd[0], skip_header=4, skip_footer= 35, delimiter='\s*',delim_whitespace=True)

In [183]:
from pandas import *
test =read_table(mwdd[0], sep='\s+\s+', skiprows=[0,1,2,3,8,9], header=None, dtype=None)
test1= {key: value for key, value in zip(test[0], test[1])}
test1

/home/david/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


{'Effective temperature [K]:': '22814',
 'Log of surface gravity [cgs]:': '7.658',
 'Spectral type [DA or DB]:': 'DA',
 'Envelope thickness [thin or thick]:': 'thick',
 'Radius [R_Sun]:': '0.0167',
 'Mass [M_Sun]:': '0.463',
 'Luminosity [L_Sun]:': '0.06764',
 'Cooling time [Gyr]:': '0.023',
 'Gravitational redshift [km/s]:': '17.6',
 'Envelope mass ratio [log(q)]:': '-15.830',
 'Log of diffusion timecale [years]:': None,
 'Li': '-1.353',
 'Be': '-0.981',
 'B': '-1.112',
 'C': '-1.365',
 'N': '-1.556',
 'O': '-1.699',
 'F': '-1.805',
 'Ne': '-1.841',
 'Na': '-1.891',
 'Mg': '-1.410',
 'Al': '-1.387',
 'Si': '-1.453',
 'P': '-1.516',
 'S': '-1.548',
 'Cl': '-1.697',
 'Ar': '-1.816',
 'K': '-1.872',
 'Ca': '-1.627',
 'Sc': '-1.597',
 'Ti': '-1.610',
 'V': '-1.661',
 'Cr': '-1.688',
 'Mn': '-1.713',
 'Fe': '-1.731',
 'Co': '-1.782',
 'Ni': '-1.784',
 'Cu': '-1.821'}

In [184]:
#np.genfromtxt(mwdd[0], delimiter='\s+\s+')

In [185]:
for row in wdtab:
    #row = np.array(row)
    teff, logg = row['Teff'], row['logg']
    if teff < 30000:
        if logg < 7.5:
            logg = 7.5
        atm =read_table('mwdd_params/{:0<5}_{}_DA_thick'.format(logg, teff), sep='\s+\s+', skiprows=[0,1,2,3,8,9], header=None, dtype=None)
        atm= {key: value for key, value in zip(atm[0], atm[1])}
        print(float(atm['Mass [M_Sun]:']), float(atm['Envelope mass ratio [log(q)]:']), float(atm['Si']), float(atm['S']), float(atm['O']), float(atm['S']))
   # else: 
    #    print('gap')

/home/david/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


0.511 -16.343 -1.997 -2.272 -2.135 -2.272
0.424 -15.4 -0.913 -1.069 -1.024 -1.069
0.464 -15.894 -1.515 -1.609 -1.758 -1.609
0.452 -15.541 -1.068 -1.232 -1.195 -1.232
0.463 -15.83 -1.453 -1.548 -1.699 -1.548
0.399 -15.65 -1.25 -1.344 -1.496 -1.344
0.404 -15.587 -1.19 -1.288 -1.439 -1.288
0.49 -15.996 -1.622 -1.719 -1.862 -1.719
0.389 -15.788 -1.366 -1.544 -1.581 -1.544


In [186]:
wdtab

Target,Teff,e_Teff,logg,e_logg,si,si_e,c,c_e,o,o_e,s,s_e
str11,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
V-UZ-SEX,17146,15,7.801,0.007,-7.08,0.03,-6.21,0.05,-6.44,0.07,-7.36,0.3
V-LM-COM,28878,1122,7.358,0.1,-6.96,0.04,-5.9,0.04,-5.63,0.0,-6.96,0.0
WD-1339+606,37074,76,7.449,0.005,-7.88,0.4,-8.32,0.1,-8.05,0.4,-7.34,0.8
WD-1504+546,21730,53,7.67,0.01,-7.13,0.2,-8.25,0.09,-8.15,0.3,-7.43,0.6
WD-2317+268,28403,19,7.585,0.005,-6.78,0.03,-5.82,0.03,-4.66,0.1,-6.65,0.2
WD-1436-216,22814,38,7.658,0.01,-7.19,0.05,-6.27,0.1,-5.92,0.4,-8.18,0.1
WD-1458+171,21406,4,7.445,0.0003,-6.19,0.3,-5.09,0.04,-5.96,0.0,-6.51,0.0
WD2257+162,22886,33,7.24,0.008,-7.38,0.05,-6.27,0.05,-8.03,0.5,-7.34,0.1
WD2256+249,21488,17,7.73,0.006,-6.88,0.02,-5.76,0.02,-5.72,0.05,-7.39,0.2


In [187]:
wdtab2 = Table.read('tables/pceb_fit_results_tdiff.csv')
wdtab2

Target,Teff,e_Teff,logg,e_logg,si,si_e,c,c_e,o,o_e,s,s_e,mass,q,tdiff_si,tdiff_c,tdiff_o,tdiff_s
str11,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
V-UZ-SEX,17146,15,7.801,0.007,-7.08,0.03,-6.21,0.05,-6.44,0.07,-7.36,0.3,0.511,-16.343,-1.997,-2.272,-2.135,-2.272
V-LM-COM,28878,1122,7.358,0.1,-6.96,0.04,-5.9,0.04,-5.63,0.0,-6.96,0.0,0.424,-15.4,-0.913,-1.069,-1.024,-1.069
WD-1339+606,37074,76,7.449,0.005,-7.88,0.4,-8.32,0.1,-8.05,0.4,-7.34,0.8,0.0,0.0,0.0,0.0,0.0,0.0
WD-1504+546,21730,53,7.67,0.01,-7.13,0.2,-8.25,0.09,-8.15,0.3,-7.43,0.6,0.464,-15.894,-1.515,-1.609,-1.758,-1.609
WD-2317+268,28403,19,7.585,0.005,-6.78,0.03,-5.82,0.03,-4.66,0.1,-6.65,0.2,0.452,-15.541,-1.068,-1.232,-1.195,-1.232
WD-1436-216,22814,38,7.658,0.01,-7.19,0.05,-6.27,0.1,-5.92,0.4,-8.18,0.1,0.463,-15.83,-1.453,-1.548,-1.699,-1.548
WD-1458+171,21406,4,7.445,0.0003,-6.19,0.3,-5.09,0.04,-5.96,0.0,-6.51,0.0,0.399,-15.65,-1.25,-1.344,-1.496,-1.344
WD2257+162,22886,33,7.24,0.008,-7.38,0.05,-6.27,0.05,-8.03,0.5,-7.34,0.1,0.404,-15.587,-1.19,-1.288,-1.439,-1.288
WD2256+249,21488,17,7.73,0.006,-6.88,0.02,-5.76,0.02,-5.72,0.05,-7.39,0.2,0.49,-15.996,-1.622,-1.719,-1.862,-1.719


In [202]:
els = ['si', 'c', 'o', 's']
el_mass = [28.085, 12.011, 15.999, 32.06]
for row in wdtab2:
    if row['Teff'] < 30000:
    #    print(row['Target'])
        #mass = np.log10((row['mass'] * u.M_sun).to(u.g).value)
        mass = row['mass']
       # print(mass)
        q = row['q']
        #print(10**(mass+q))
        fluxes = []
        errors  = []
        for i, el in enumerate(els):
            rate = row[el] + np.log10(mass) + q +np.log10(el_mass[i]/1.008)- row['tdiff_{}'.format(el)]
            rate_cgs = ((10**rate)*u.M_sun/u.yr).to(u.g/u.s)
            fluxes.append(rate_cgs.value) 
            #print(el, rate_cgs)
            if row['{}_e'.format(el)] > 0:
                error = rate_cgs * ((row['e_Teff']/row['Teff'])**2 + (row['e_logg']/row['logg'])**2 + 2.303*row['{}_e'.format(el)]**2)**0.5
            else:
                error = 0*u.g/u.s
            #print(error)
            errors.append(error.value)
        out = ''
        for f, e in zip(fluxes, errors):
            out = out + '{},{},'.format(f, e)
        print(out)
            


336385.09132898465,15320.376491126106,2008822.1030577705,152446.57686140286,1149357.5412698414,122104.13677743055,379599.67271970556,172820.67552643747,
265938.7946240172,19504.520518481393,1870193.032722438,137163.96072196623,4182154.8214478693,0.0,434781.2289431927,0.0,
252309.62605658156,76582.37195963145,10163.389241779822,1388.4078297107815,24018.851115564787,10935.24372006461,179235.03695939758,163201.11907154648,
443153.59234975616,20179.708286115194,2521500.234864761,114820.5499432861,44583348.568265304,6765937.3132430585,995504.8128374042,302149.8672373404,
220291.03855318,16721.78697637147,975217.6867894628,148009.9020286767,4117325.865678967,2499337.1277615083,32024.762688801176,4860.434805756743,
1800481.7973469386,819703.8918181054,12036313.025935011,730638.5580566927,3069072.482069326,0.0,1221455.9934990634,0.0,
118521.51780564559,8995.775491158529,818280.0127263956,62107.399733606566,26817.33484414485,20348.567102311365,185902.9555083932,28213.981520061177,
479304.955777

In [189]:
for row in wdtab2:
    if row['Target'] == 'WD0710+741':
        print(row['Target'])
        #mass = np.log10((row['mass'] * u.M_sun).to(u.g).value)
        mass = row['mass']
       # print(mass)
        q = row['q']
        #print(10**(mass+q))
        for el in els:
            rate = row[el] + np.log10(mass) +np.log10(28) + q - row['tdiff_{}'.format(el)]
            #print(rate)
            print(el, ((10**rate)*u.M_sun/u.yr).to(u.g/u.s))
            rate = (mass*u.Msun * 10**q * 10**row[el]/((10**row['tdiff_{}'.format(el)])*u.yr)).to(u.g/u.s)
            print(el, rate)
            #print(el, np.log10(((10**rate)*u.g/u.yr).to(u.g/u.s).value))

WD0710+741
si 284780.0591845519 g / s
si 10170.716399448254 g / s
c 7286342.40301745 g / s
c 260226.51439348073 g / s
o 6158991.091865036 g / s
o 219963.96756660924 g / s
s 470445.7786702775 g / s
s 16801.63495250987 g / s
